In [1]:
import os, sys
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns 

from os.path import join 
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from IPython.display import display_html

import warnings
warnings.filterwarnings("ignore")

In [2]:
DataPath = r'D:\GitWork\titanic\data'
OutPath  = r'D:\GitWork\titanic\output'

csv_train = 'train.csv'
csv_test  = 'test.csv'

In [3]:
f_abspath = join(DataPath, csv_train)
df_train = pd.read_csv(f_abspath)

display(df_train.head())
display(df_train.info())
# display(df_train.describe())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

In [4]:
f_abspath = join(DataPath, csv_test)
df_test = pd.read_csv(f_abspath)

display(df_test.head())
display(df_test.info())

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


None

In [5]:
df_data = df_train.append(df_test)
len_train = len(df_train)
print('Num rows for train:', len_train)

display(df_data.head())
# display(df_data.describe())
display(df_data.info())

Num rows for train: 891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


None

In [6]:
# For Sex
le_sex = LabelEncoder()
df_data['SexCode'] = le_sex.fit_transform(df_data.Sex)
display(df_data.head())
display(df_data.info())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexCode
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  SexCode      1309 non-null   int32  
dtypes: float64(3), int32(1), int64(4), object(5)
memory usage: 138.1+ KB


None

In [7]:
# For Fare

# Fill fare null valeu
mean_fare = df_data.Fare.mean()
df_data.Fare.fillna(mean_fare, inplace=True)
display(df_data.info())

def classes2dict(le):
    return {
        le.transform([x])[0]: x for x in le.classes_
    }

le_fb5 = LabelEncoder()
df_data['FareBin_5'] = pd.qcut(df_data.Fare, 5)
df_data['FareBinCode_5'] = le_fb5.fit_transform(df_data.FareBin_5)
df_data.drop(['FareBin_5'], axis=1, inplace=True)
display(df_data.head())

dict_fb5 = classes2dict(le_fb5)
print('Dict fare bin:'); display(dict_fb5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  SexCode      1309 non-null   int32  
dtypes: float64(3), int32(1), int64(4), object(5)
memory usage: 138.1+ KB


None

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexCode,FareBinCode_5
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,4
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,4
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1


Dict fare bin:


{0: Interval(-0.001, 7.854, closed='right'),
 1: Interval(7.854, 10.5, closed='right'),
 2: Interval(10.5, 21.679, closed='right'),
 3: Interval(21.679, 41.579, closed='right'),
 4: Interval(41.579, 512.329, closed='right')}

In [8]:
# Family Size
df_data['FamilySize'] = df_data.SibSp + df_data.Parch + 1
display(df_data.head())
display(df_data.info())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexCode,FareBinCode_5,FamilySize
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,4,2
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,4,2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    1309 non-null   int64  
 1   Survived       891 non-null    float64
 2   Pclass         1309 non-null   int64  
 3   Name           1309 non-null   object 
 4   Sex            1309 non-null   object 
 5   Age            1046 non-null   float64
 6   SibSp          1309 non-null   int64  
 7   Parch          1309 non-null   int64  
 8   Ticket         1309 non-null   object 
 9   Fare           1309 non-null   float64
 10  Cabin          295 non-null    object 
 11  Embarked       1307 non-null   object 
 12  SexCode        1309 non-null   int32  
 13  FareBinCode_5  1309 non-null   int32  
 14  FamilySize     1309 non-null   int64  
dtypes: float64(3), int32(2), int64(5), object(5)
memory usage: 153.4+ KB


None

In [9]:
# aggregate target column values 
def aggrColValues(df_src, col_aggr, col_targ, col_outs):
    tmp_list = []
    for tk in df_src[col_aggr].unique():
        tmp = df_src.loc[df_src[col_aggr]==tk, col_targ]
        if tmp.count() > 1:
            tmp_list.append(df_src.loc[df_src[col_aggr]==tk, col_outs])
    return pd.concat(tmp_list)
   
col_selected = ['PassengerId', 'Name', 'Ticket', 'Fare', 'FamilySize', 'Survived']
df_tmp = aggrColValues(df_data, 'Ticket', 'PassengerId', col_selected)
display(df_tmp.head(15))

# Show dead and alive people that have same ticket
df_alone = df_tmp.loc[df_tmp.FamilySize == 1]
df_group = df_tmp.loc[df_tmp.FamilySize >  1]
display(df_alone[df_alone.Survived.notnull()].head())
display(df_group[df_group.Survived.notnull()].head())

num_total = len(df_tmp)
print('\nKeep same ticket:', len(df_tmp))

num_alone = len(df_alone)
print('Alone: {} ({}%)'.format(num_alone, round(num_alone/num_total, 3)))

num_group = len(df_group)
print('Group: {} ({}%)'.format(num_group, round(num_group/num_total, 3)))

,PassengerId,Name,Ticket,Fare,FamilySize,Survived
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",PC 17599,71.2833,2,1.0
234,1126,"Cumings, Mr. John Bradley",PC 17599,71.2833,2,NaN
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",113803,53.1000,2,1.0
137,138,"Futrelle, Mr. Jacques Heath",113803,53.1000,2,0.0
6,7,"McCarthy, Mr. Timothy J",17463,51.8625,1,0.0
146,1038,"Hilliard, Mr. Herbert Henry",17463,51.8625,1,NaN
7,8,"Palsson, Master. Gosta Leonard",349909,21.0750,5,0.0
24,25,"Palsson, Miss. Torborg Danira",349909,21.0750,5,0.0
374,375,"Palsson, Miss. Stina Viola",349909,21.0750,5,0.0
567,568,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",349909,21.0750,5,0.0


,PassengerId,Name,Ticket,Fare,FamilySize,Survived
6,7,"McCarthy, Mr. Timothy J",17463,51.8625,1,0.0
20,21,"Fynney, Mr. Joseph J",239865,26.0000,1,0.0
791,792,"Gaskell, Mr. Alfred",239865,26.0000,1,0.0
195,196,"Lurette, Miss. Elise",PC 17569,146.5208,1,1.0
681,682,"Hassab, Mr. Hammad",PC 17572,76.7292,1,1.0


,PassengerId,Name,Ticket,Fare,FamilySize,Survived
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",PC 17599,71.2833,2,1.0
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",113803,53.1000,2,1.0
137,138,"Futrelle, Mr. Jacques Heath",113803,53.1000,2,0.0
7,8,"Palsson, Master. Gosta Leonard",349909,21.0750,5,0.0
24,25,"Palsson, Miss. Torborg Danira",349909,21.0750,5,0.0



Keep same ticket: 596
Alone: 127 (0.213%)
Group: 469 (0.787%)


In [10]:
# For connected_survival 
#
# Set connected survival is 0.5.
# If there are someone alive in same ticket group, set connected survival is 1
# If nobody alive in same ticket group, set connected survival is 0

col_selected = ['PassengerId', 'Survived', 'Ticket']
df_conn = df_data[col_selected]
df_conn['Connected_Survival'] = 0.5

for tk, df_grp in df_conn.groupby('Ticket'):
    if (len(df_grp) > 1):
        for idx, row in df_grp.iterrows():
            smax = df_grp.drop(idx).Survived.max()
            smin = df_grp.drop(idx).Survived.min()
            passId = row.PassengerId
            if (smax == 1.0):
                df_conn.loc[df_conn.PassengerId == passId, 'Connected_Survival'] = 1
            elif (smax == 0.0):
                df_conn.loc[df_conn.PassengerId == passId, 'Connected_Survival'] = 0

display(df_conn.head())
                
# print
df_same_tk = df_conn[df_conn.groupby('Ticket')['PassengerId'].transform('count') > 1]
df_same_tk.sort_values(by=['Ticket', 'PassengerId'], inplace=True)
display(df_same_tk.head())
print('People keep same ticket: {}'.format(len(df_same_tk)))

num_conn_surv = df_conn[df_conn.Connected_Survival != 0.5]['Connected_Survival'].count()
print('People have connected infomation: {}'.format(num_conn_surv))

# The relation between Connected_Survival and Survived: 1
df_tmp = df_conn.groupby('Connected_Survival')[['Survived']].mean().round(3)
display(df_tmp)

# The relation between Connected_Survival and Survived: 2
kwagrs = { 'margins': True, 'margins_name': 'Total' }
ct = pd.crosstab(df_conn.Connected_Survival, df_conn.Survived, **kwagrs)
ct['S_Rate'] = (ct[1.0]/ ct.Total).round(3)
display(ct)

,PassengerId,Survived,Ticket,Connected_Survival
0,1,0.0,A/5 21171,0.5
1,2,1.0,PC 17599,0.5
2,3,1.0,STON/O2. 3101282,0.5
3,4,1.0,113803,0.0
4,5,0.0,373450,0.5


,PassengerId,Survived,Ticket,Connected_Survival
257,258,1.0,110152,1.0
504,505,1.0,110152,1.0
759,760,1.0,110152,1.0
262,263,0.0,110413,1.0
558,559,1.0,110413,1.0


People keep same ticket: 596
People have connected infomation: 496


,Survived
Connected_Survival,
0.0,0.225
0.5,0.298
1.0,0.728


Survived,0.0,1.0,Total,S_Rate
Connected_Survival,,,,
0.0,110,32,142,0.225
0.5,384,163,547,0.298
1.0,55,147,202,0.728
Total,549,342,891,0.384


In [11]:
# Training
col_selected = ['PassengerId', 'Survived', 'Sex', 'Pclass', 'SibSp', 'Parch', 'Ticket', 'Fare']
ds_tran = df_train[col_selected]
display(ds_tran.head())

def classes2dict(encoder):
    return {
        encoder.transform([x])[0]: x for x in encoder.classes_
    }

# Transform sex label to number
le_sex = LabelEncoder()
ds_tran['SexCode'] = le_sex.fit_transform(ds_tran.Sex)
# display(ds_tran.head())

dict_sex = classes2dict(le_sex)
print('Dict sex:'); display(dict_sex)

# Split Fare to five parts
ds_tran['FareBin_5'] = pd.qcut(ds_tran.Fare, 5)

le_fb5 = LabelEncoder()
ds_tran['FareBinCode_5'] = le_fb5.fit_transform(ds_tran.FareBin_5)

dict_fb5 = classes2dict(le_fb5)
print('Dict fare bin:'); display(dict_fb5)

ds_tran.drop(['FareBin_5'], axis=1, inplace=True)
# display(ds_tran.head())

# Create connected_survival 
ds_tran['FamilySize'] = ds_tran.SibSp + ds_tran.Parch + 1
ds_tran['Connected_Survival'] = 0.5

for tk, df_grp in ds_tran.groupby('Ticket'):
    if (len(df_grp) > 1):
        for idx, row in df_grp.iterrows():
            smax = df_grp.drop(idx).Survived.max()
            smin = df_grp.drop(idx).Survived.min()
            passId = row.PassengerId
            if (smax == 1.0):
                ds_tran.loc[ds_tran.PassengerId == passId, 'Connected_Survival'] = 1
            elif (smax == 0.0):
                ds_tran.loc[ds_tran.PassengerId == passId, 'Connected_Survival'] = 0

display(ds_tran.head())
                
# print
print('People keep same ticket: {}'.format(len(ds_tran)))

num_conn = ds_tran[ds_tran.Connected_Survival != 0.5]['Connected_Survival'].count()
print('People have connected infomation: {}'.format(num_conn))

ct = pd.crosstab(ds_tran.Connected_Survival, ds_tran.Survived, margins = True, margins_name='Total')
ct['S_Rate'] = (ct[1]/ ct.Total).round(3)
display(ct)

# Start training
features = ['SexCode', 'Pclass', 'FareBinCode_5', 'Connected_Survival']
inps = ds_tran.filter(features, axis=1)
lbls = ds_tran.Survived

model_conn = RandomForestClassifier(random_state=2, n_estimators=250, min_samples_split=20, oob_score=True)
model_conn.fit(inps, lbls)
print('oob score :%f' %(model_conn.oob_score_))

,PassengerId,Survived,Sex,Pclass,SibSp,Parch,Ticket,Fare
0,1,0,male,3,1,0,A/5 21171,7.2500
1,2,1,female,1,1,0,PC 17599,71.2833
2,3,1,female,3,0,0,STON/O2. 3101282,7.9250
3,4,1,female,1,1,0,113803,53.1000
4,5,0,male,3,0,0,373450,8.0500


Dict sex:


{0: 'female', 1: 'male'}

Dict fare bin:


{0: Interval(-0.001, 7.854, closed='right'),
 1: Interval(7.854, 10.5, closed='right'),
 2: Interval(10.5, 21.679, closed='right'),
 3: Interval(21.679, 39.688, closed='right'),
 4: Interval(39.688, 512.329, closed='right')}

,PassengerId,Survived,Sex,Pclass,SibSp,Parch,Ticket,Fare,SexCode,FareBinCode_5,FamilySize,Connected_Survival
0,1,0,male,3,1,0,A/5 21171,7.2500,1,0,2,0.5
1,2,1,female,1,1,0,PC 17599,71.2833,0,4,2,0.5
2,3,1,female,3,0,0,STON/O2. 3101282,7.9250,0,1,1,0.5
3,4,1,female,1,1,0,113803,53.1000,0,4,2,0.0
4,5,0,male,3,0,0,373450,8.0500,1,1,1,0.5


People keep same ticket: 891
People have connected infomation: 344


Survived,0,1,Total,S_Rate
Connected_Survival,,,,
0.0,110,32,142,0.225
0.5,384,163,547,0.298
1.0,55,147,202,0.728
Total,549,342,891,0.384


oob score :0.821549


In [12]:
# Prediction
features = ['PassengerId', 'Sex', 'Pclass', 'SibSp', 'Parch', 'Ticket', 'Fare']
ds_pred = df_test[features]
display(ds_pred.head())
display(ds_pred.info())

# For SexCode
le_sex = LabelEncoder()
ds_pred['SexCode'] = le_sex.fit_transform(ds_pred.Sex)
# display(ds_pred.head())

# Fill null fare value
ds_pred.Fare.fillna(ds_pred.Fare.mean(), inplace=True)

# For FareBin5
le_fb5 = LabelEncoder()
ds_pred['FareBin_5'] = pd.qcut(ds_pred.Fare, 5)
ds_pred['FareBinCode_5'] = le_fb5.fit_transform(ds_pred['FareBin_5'])
ds_pred.drop(['FareBin_5'], axis=1, inplace=True)
# display(ds_pred.head())

# For Connected_Survival
cols_conn = ['PassengerId', 'Survived', 'SibSp', 'Parch', 'Ticket']
df_conn = df_data[cols_conn]
df_conn['FamilySize'] = ds_pred.SibSp + ds_pred.Parch + 1
df_conn['Connected_Survival'] = 0.5
# display(df_conn)

for tk, df_grp in df_conn.groupby('Ticket'):
    if len(df_grp) > 1:
        for idx, row in df_grp.iterrows():
            smax = df_grp.drop(idx).Survived.max()
            smin = df_grp.drop(idx).Survived.min()
            passId = row.PassengerId
            if (smax == 1.0):
                df_conn.loc[df_conn.PassengerId == passId, 'Connected_Survival'] = 1
            elif (smax == 0.0):
                df_conn.loc[df_conn.PassengerId == passId, 'Connected_Survival'] = 0

ds_pred['Connected_Survival'] = df_conn[df_conn.PassengerId.isin(df_test.PassengerId)]['Connected_Survival']
display(ds_pred)

# print
print('People keep same ticket: {}'.format(len(ds_pred)))

num_conn = ds_pred[ds_pred.Connected_Survival != 0.5]['Connected_Survival'].count()
print('People have connected infomation: {}'.format(num_conn))

col_selected = ['SexCode', 'Pclass', 'FareBinCode_5', 'Connected_Survival']
inps = ds_pred[col_selected]
ds_pred['Survived'] = model_conn.predict(inps)
display(ds_pred.head())

# Save predict result to xlsx
col_selected = ['PassengerId', 'Survived']
f_abspath = join(OutPath, 'Submission_v3.csv')
ds_pred[col_selected].to_csv(f_abspath, index=False)

print('\ndone')

,PassengerId,Sex,Pclass,SibSp,Parch,Ticket,Fare
0,892,male,3,0,0,330911,7.8292
1,893,female,3,1,0,363272,7.0000
2,894,male,2,0,0,240276,9.6875
3,895,male,3,0,0,315154,8.6625
4,896,female,3,1,1,3101298,12.2875


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Sex          418 non-null    object 
 2   Pclass       418 non-null    int64  
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   Ticket       418 non-null    object 
 6   Fare         417 non-null    float64
dtypes: float64(1), int64(4), object(2)
memory usage: 23.0+ KB


None

,PassengerId,Sex,Pclass,SibSp,Parch,Ticket,Fare,SexCode,FareBinCode_5,Connected_Survival
0,892,male,3,0,0,330911,7.8292,1,1,0.5
1,893,female,3,1,0,363272,7.0000,0,0,0.5
2,894,male,2,0,0,240276,9.6875,1,1,0.5
3,895,male,3,0,0,315154,8.6625,1,1,0.5
4,896,female,3,1,1,3101298,12.2875,0,2,1.0
...,...,...,...,...,...,...,...,...,...,...
413,1305,male,3,0,0,A.5. 3236,8.0500,1,1,0.5
414,1306,female,1,0,0,PC 17758,108.9000,0,4,1.0
415,1307,male,3,0,0,SOTON/O.Q. 3101262,7.2500,1,0,0.5
416,1308,male,3,0,0,359309,8.0500,1,1,0.5


People keep same ticket: 418
People have connected infomation: 152


,PassengerId,Sex,Pclass,SibSp,Parch,Ticket,Fare,SexCode,FareBinCode_5,Connected_Survival,Survived
0,892,male,3,0,0,330911,7.8292,1,1,0.5,0
1,893,female,3,1,0,363272,7.0000,0,0,0.5,1
2,894,male,2,0,0,240276,9.6875,1,1,0.5,0
3,895,male,3,0,0,315154,8.6625,1,1,0.5,0
4,896,female,3,1,1,3101298,12.2875,0,2,1.0,1



done
